In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# %matplotlib inline

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available", len(physical_devices))

Num GPUs Available 1


In [3]:
# path to dataset
directory = "./FACD_image"
classes = ["1977", "Amaro", "Apollo", "Brannan", "Earlybird"]
target_size = (224,224)
seed = 42;
batch_size=100

# create a image generator for keras, that can load images batchwise
data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, validation_split=0.2)

train_batches = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size, classes=classes, color_mode='rgb', batch_size=batch_size, shuffle=True, seed=seed,
    subset='training'
)

valid_batches = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size, classes=classes,  color_mode='rgb', batch_size=batch_size, shuffle=True, seed=seed, 
    subset='validation'
)    



Found 5120 images belonging to 5 classes.
Found 1280 images belonging to 5 classes.


In [4]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(224,224,3)), 
    MaxPool2D(pool_size=(2, 2), strides=2), 
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'), 
    MaxPool2D(pool_size=(2,2), strides=2), 
    Flatten(), 
    Dense(units=len(classes), activation='softmax'),
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1003525   
Total params: 1,022,917
Trainable params: 1,022,917
Non-trainable params: 0
______________________________________________

In [6]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 52 steps, validate for 13 steps
Epoch 1/10
52/52 [==============================] - 102s 2s/step - loss: 49.1419 - accuracy: 0.4439 - val_loss: 7.1513 - val_accuracy: 0.6359
Epoch 2/10
52/52 [==============================] - 92s 2s/step - loss: 3.4897 - accuracy: 0.7441 - val_loss: 2.2340 - val_accuracy: 0.8047
Epoch 3/10
52/52 [==============================] - 92s 2s/step - loss: 1.3678 - accuracy: 0.8248 - val_loss: 1.7683 - val_accuracy: 0.8148
Epoch 4/10
52/52 [==============================] - 95s 2s/step - loss: 0.9477 - accuracy: 0.8650 - val_loss: 1.9079 - val_accuracy: 0.8133
Epoch 5/10
52/52 [==============================] - 93s 2s/step - loss: 0.9234 - accuracy: 0.8734 - val_loss: 1.4783 - val_accuracy: 0.8422
Epoch 6/10
52/52 [==============================] - 89s 2s/step - loss: 0.4905 - accuracy: 0.9139 - val_loss: 1.5007 - val_accuracy: 0.8391
Epoch 7/10
52/52 [==============================] - 89s 2s/step - 

In [8]:
filepath = "./keras_five_filters_color_few_layers.hdf5"

model.save(
   filepath, overwrite=True, include_optimizer=True, save_format="h5",
    signatures=None, options=None
)